In [1]:
import json
import pickle

In [2]:
def extract_names(s):
    name = ''
    sentence = ''
    flag = 0
    for w in s:
        if w != ':' and flag == 0:
            name += w
        if w == ':':
            flag = 1
        elif flag == 1:
            sentence += w
    return name, sentence

In [3]:
def read_conversation(data):
    conversations = []

    for i in range(0, len(data)):
        if len(data[i]['dialogue'].split('\r\n')) > 1:
            sentences = data[i]['dialogue'].replace("|", " ").split('\r\n')
            
        else:
            sentences = data[i]['dialogue'].replace("|", " ").split('\n')
            
        if len(sentences) == 1:
            continue
            
        conversations.append(sentences)
    
    return conversations

In [4]:
import re
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
import allennlp_models.structured_prediction
import os
import spacy
import neuralcoref

nlp = spacy.load("en")
# Add neural coref to SpaCy's pipe

neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
def coreference(s):
    doc = nlp(s)
    return doc._.coref_clusters

In [5]:
def find_nearest_coref(full_conv, index, word):
    ref_list = ['he', 'she', 'him', 'her', 'it', 'his']
    prev_sentence = ''
    for i in range(index, -1, -1):
        #print(i)
        cur_sentence =  full_conv[i][1]
        prev_sentence = cur_sentence + ' ' + prev_sentence
        coref = coreference(prev_sentence)
        #print('----')
        #print(prev_sentence)
        #print(coref)
        #print('----')
        if len(coref) > 0:
            for clusters in coref:
                if word in [str(w) for w in clusters.mentions] and str(clusters.main).lower() not in ref_list:
                    return str(clusters.main)
    
    return word

In [6]:
from nltk.tokenize import word_tokenize
import string
punc = string.punctuation

In [7]:
first = ['i', 'me', 'my']
second = ['you', 'u', 'your']
third = ['he', 'she', 'him', 'her', 'it', 'his']

def transform_perspective(conv):
    transformed_conv = ''
    temp = []
    flag = 0
    
    
    for i in range(0, len(conv)):
        #print(i)
        cur_name = conv[i][0]
        cur_uttr = conv[i][1]
        s = ''
        

        for word in word_tokenize(cur_uttr):
            if word.lower() in first:
                temp_word = cur_name
            elif word.lower() in second:
                try:
                    temp_word = conv[i+1][0]
                except:
                    temp_word = conv[i-1][0]
            elif word.lower() in third:
                temp_word = find_nearest_coref(conv, i, word)
            else:
                temp_word = word

            s = s + temp_word + ' '

        #j = len(s)-1
        #while s[j] == ' ':
        #    j = j - 1

        #if s[j] not in punc:
        s += '.'


        temp.append(s)
                
                
        
    transformed_conv = ' '.join(temp)

    return transformed_conv, temp

In [8]:
import re

alphabets = "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu)"

def sentence_tokenize(text):
    text = " " + text + "  "
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text:
        text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms+" "+starters, "\\1<stop> \\2", text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" +
                  alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>", text)
    text = re.sub(alphabets + "[.]" + alphabets +
                  "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(" "+suffixes+"[.] "+starters, " \\1<stop> \\2", text)
    text = re.sub(" "+suffixes+"[.]", " \\1<prd>", text)
    text = re.sub(" " + alphabets + "[.]", " \\1<prd>", text)

    text = re.sub("([0-9])" + "[.]" + "([0-9])", "\\1<prd>\\2", text)

    if "..." in text:
        text = text.replace("...", "<prd><prd><prd>")
    if "”" in text:
        text = text.replace(".”", "”.")
    if "\"" in text:
        text = text.replace(".\"", "\".")
    if "!" in text:
        text = text.replace("!\"", "\"!")
    if "?" in text:
        text = text.replace("?\"", "\"?")

    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")

    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [9]:
from tqdm import tqdm
def transformat(name):
    with open(name + '.json', encoding = 'utf8') as json_file:
        data = json.load(json_file)
    convs = read_conversation(data)
    processed_convs = []
    for i in range(0, len(convs)):
        temp = []
        conv = convs[i]
        name_prev = None
        same_sentence = ''
        for j in range(0, len(conv)):
            name, sentence = extract_names(conv[j])

            if name != name_prev:
                if name_prev is not None:
                    temp.append([name_prev, same_sentence])
                name_prev = name
                same_sentence = sentence
            elif name == name_prev:
                same_sentence = same_sentence + ' . ' + sentence

        temp.append([name_prev, same_sentence])

        processed_convs.append(temp)
        
    transformed_conv = []
    for j in tqdm(range(0, len(processed_convs))):
        transformed_conv.append(transform_perspective(processed_convs[j])[1])
    
    return transformed_conv
        
        
    

In [10]:
transformed_conv = transformat('test')

100%|██████████| 819/819 [06:38<00:00,  2.05it/s]


In [14]:
with open('transformed_conv.pkl', 'wb') as f:
    pickle.dump(transformed_conv, f)

In [15]:
def compress_triple(triples):
    temp_set = []
    for i in range(0, len(triples)):
        cur = triples[i]
        cur_subject = cur['subject']
        cur_relation = cur['relation']
        cur_object = cur['object']
        
        if len(temp_set) == 0:
            temp_set.append([cur_subject, cur_relation, cur_object])
        else:
            flag = 0
            #print(temp_set)
            for j in range(0, len(temp_set)):
                
                if temp_set[j][0] == cur_subject and temp_set[j][1] == cur_relation:
                    
                    if len(cur_object) > len(temp_set[j][2]):
                        temp_set[j][2] = cur_object
                    flag = 1
                    
                elif temp_set[j][0] == cur_subject and temp_set[j][2] == cur_object:
                    if len(cur_relation) > len(temp_set[j][1]):
                        temp_set[j][1] = cur_relation
                    flag = 1
                    
                elif temp_set[j][2] == cur_object and temp_set[j][1] == cur_relation:
                    if len(cur_subject) > len(temp_set[j][0]):
                        temp_set[j][0] = cur_subject
                    flag = 1
                    
            
            if flag == 0:
                temp_set.append([cur_subject, cur_relation, cur_object])
                    
                        
    return temp_set
            

In [18]:
from openie import StanfordOpenIE
with StanfordOpenIE(be_quiet=False) as client:
    tuple_set = []
    for i in tqdm(range(0, len(transformed_conv))):
        triples = []
        temp_set = []
        for uttr in transformed_conv[i]:
            sentences = sentence_tokenize(uttr)
            for sent in sentences:
                if "?" not in sent:
                    #print(sent)
                    triple =  client.annotate(sent,properties={
    'timeout': '5000000',
    'annotators': 'tokenize,ssplit,pos,depparse,parse',
    'outputFormat': 'json'
})
                    if len(triple) > 0:
                        triples.extend(compress_triple(triple))
                        
        tuple_set.append(triples)

  0%|          | 0/819 [00:00<?, ?it/s]


TypeError: annotate() got an unexpected keyword argument 'be_quiet'

In [ ]:
with open('test_raw_triples.pkl', 'wb') as f:
    pickle.dump(tuple_set, f)

In [ ]:
action_input = []
action_adj = []

In [ ]:
node_num = []

for i in range(len(tuple_set)):
    node_set = set()
    if len(tuple_set[i]) == 0:
        print(i)
    for u in tuple_set[i]:
        node_set.add(u[0])
        node_set.add(u[1])
        node_set.add(u[2])     
    node_num.append(len(node_set))

In [ ]:
max(node_num)

In [ ]:
import numpy as np
action_input = []
action_adj = []
for i in range(0, len(tuple_set)):
    id2node = {}
    node2id = {}
    adj_temp = np.zeros([max(node_num), max(node_num)])
    index = 0
    if len(tuple_set[i]) == 0:
        action_input.append('<pad>')
    else:
        temp_text = ''
        for u in tuple_set[i]:
            if u[0] not in node2id:
                node2id[u[0]] = index
                id2node[index] = u[0]
                
                if len(temp_text) == 0:
                    temp_text = u[0] 
                else:
                    temp_text = temp_text + '. </s><s> ' + u[0] 
                
                index = index + 1
            if u[1] not in node2id:
                node2id[u[1]] = index
                id2node[index] = u[1]
                index = index + 1
                
                if len(temp_text) == 0:
                    temp_text = u[1] 
                else:
                    temp_text = temp_text + '. </s><s> ' + u[1] 
                    
            if u[2] not in node2id:
                node2id[u[2]] = index
                id2node[index] = u[2]
                index = index + 1
                
                if len(temp_text) == 0:
                    temp_text = u[2] 
                else:
                    temp_text = temp_text + '. </s><s> ' + u[2] 
                
            adj_temp[node2id[u[0]]][node2id[u[0]]] = 1
            adj_temp[node2id[u[1]]][node2id[u[1]]] = 1
            adj_temp[node2id[u[2]]][node2id[u[2]]] = 1
            
            adj_temp[node2id[u[0]]][node2id[u[1]]] = 1
            adj_temp[node2id[u[1]]][node2id[u[0]]] = 1
            
            adj_temp[node2id[u[1]]][node2id[u[2]]] = 1
            adj_temp[node2id[u[2]]][node2id[u[1]]] = 1
            
            
    
        action_input.append(temp_text)
    action_adj.append(adj_temp)

In [ ]:
with open('test_actions.pkl', 'wb') as f:
    pickle.dump(action_input, f)

In [ ]:
with open('test_action_adj.pkl', 'wb') as f:
    pickle.dump(action_adj, f)

In [ ]:
# similarly you could follow the process to create action graph files for train/test

In [ ]:
with open('val_actions.pkl', 'wb') as f:
    pickle.dump(action_input, f)

In [ ]:
with open('val_action_adj.pkl', 'wb') as f:
    pickle.dump(action_adj, f)